### Импорт библиотек:
____

In [65]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as  plt

### Справочная информация про DataFrame:
____

 - olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей:

     1. customer_id — позаказный идентификатор пользователя (аналог номера паспорта);
     2. customer_unique_id —  уникальный идентификатор пользователя;
     3. customer_zip_code_prefix —  почтовый индекс пользователя;
     4. customer_city —  город доставки пользователя;
     5. customer_state —  штат доставки пользователя.

 - olist_orders_dataset.csv —  таблица заказов:
    1. order_id —  уникальный идентификатор заказа (номер чека);
    2. customer_id —  позаказный идентификатор пользователя (аналог номера паспорта);
    3. order_status —  статус заказа;
    4. order_purchase_timestamp —  время создания заказа;
    5. order_approved_at —  время подтверждения оплаты заказа;
    6. order_delivered_carrier_date —  время передачи заказа в логистическую службу;
    7. order_delivered_customer_date —  время доставки заказа;
    8. order_estimated_delivery_date —  обещанная дата доставки.

 - olist_order_items_dataset.csv —  товарные позиции, входящие в заказы:
    1. order_id —  уникальный идентификатор заказа (номер чека);
    2. order_item_id —  идентификатор товара внутри одного заказа;
    3. product_id —  ид товара (аналог штрихкода);
    4. seller_id — ид производителя товара;
    5. shipping_limit_date —  максимальная дата доставки продавцом для передачи заказа партнеру по логистике;
    6. price —  цена за единицу товара;
    7. freight_value —  вес товара.

### Главные задачи:
____

1. Сколько у нас пользователей, которые совершили покупку только один раз?

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

3. По каждому товару определить, в какой день недели товар чаще всего покупается.

4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть.

5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц.
6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построим RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию.

### Выгрузка данных:
____

In [66]:
olist_customers_dataset = pd.read_csv("olist_customers_dataset.csv")
olist_customers_dataset.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [67]:
olist_orders_dataset = pd.read_csv("olist_orders_dataset.csv")
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


Также предлагаю сразу все нужные колонки со временем перевести во временной формат:

In [68]:
olist_orders_dataset["order_purchase_timestamp"] = pd.to_datetime(olist_orders_dataset["order_purchase_timestamp"])
olist_orders_dataset["order_approved_at"] = pd.to_datetime(olist_orders_dataset["order_approved_at"])
olist_orders_dataset["order_delivered_customer_date"] = pd.to_datetime(olist_orders_dataset["order_delivered_customer_date"])

In [69]:
olist_order_items_dataset = pd.read_csv("olist_order_items_dataset.csv")
olist_order_items_dataset.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


### Ответ на первый вопрос:
____

In [70]:
df12 = olist_customers_dataset.merge(olist_orders_dataset,on = "customer_id")
df12.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


In [71]:
df12.query("order_status == 'canceled'").groupby("order_delivered_customer_date",as_index = False).size()

,order_delivered_customer_date,size
0,2016-10-14 15:07:11,1
1,2016-10-16 14:36:59,1
2,2016-10-19 18:47:43,1
3,2016-11-08 10:58:34,1
4,2016-11-09 14:53:50,1
5,2018-03-21 22:03:51,1


In [72]:
df12["order_status"].unique()

array(['delivered', 'canceled', 'invoiced', 'shipped', 'processing',
       'unavailable', 'approved', 'created'], dtype=object)

Фильтр нам стоит поставить на order_id по условию задания и стоит отфильтровать также по статусу заказу: оставить все те, кроме canceled и unavailable, так как остальные заказы уже прошли стадию совершения покупки.

In [73]:
answer1 = df12.query("order_status != 'canceled' and order_status != 'unavailable'")

In [74]:
answer1 = answer1.groupby("customer_unique_id",as_index=False).aggregate({"order_id":"count"}).\
    query("order_id == 1").rename(columns = {"order_id":"quantity_of_orders"}).shape[0]
answer1

92102

Итого: пользователей, которые совершили лишь одну покупку - 92102.

### Ответ на второй вопрос :
____


Сначала хотелось бы посмотреть на уникальные значения, которые у нас в принципе есть :

In [75]:
df12["order_status"].unique()

array(['delivered', 'canceled', 'invoiced', 'shipped', 'processing',
       'unavailable', 'approved', 'created'], dtype=object)

В наших данных у нас наблюдается всего 2 причины по которым наши товары не могут быть доставлены юзерам и так как у нас информация как будто идёт в реальном времени, здесь существуют всего 2 колонки:
1. canceled;
2. unavailable.

А теперь отберём нужные нам колонки и на всяких случай также укажем, что order_delivered_customer_date должны быть NaN значения

In [76]:
undelivered = df12.query("order_delivered_customer_date == 'NaN' and order_status in ('unavailable','canceled')")

А теперь отберём только те заказы, которые были подтверждённые:

In [77]:
undelivered = undelivered.groupby(["order_approved_at","order_status"],as_index = False).size()
undelivered.head()

,order_approved_at,order_status,size
0,2016-10-04 10:26:40,canceled,1
1,2016-10-05 08:45:09,canceled,1
2,2016-10-06 14:22:19,unavailable,1
3,2016-10-06 15:50:56,canceled,1
4,2016-10-06 15:54:47,canceled,1


Далее добавим колонку с месяцами :

In [78]:
undelivered["months"] = undelivered["order_approved_at"].apply(lambda x:x.strftime("%m-%Y"))
undelivered.head()

,order_approved_at,order_status,size,months
0,2016-10-04 10:26:40,canceled,1,10-2016
1,2016-10-05 08:45:09,canceled,1,10-2016
2,2016-10-06 14:22:19,unavailable,1,10-2016
3,2016-10-06 15:50:56,canceled,1,10-2016
4,2016-10-06 15:54:47,canceled,1,10-2016


После произведём группировку по месяцу и статусу заказа, а далее совершим группировку по статусу и посчитаем среднее в месяц и также округлим наши значения :

In [79]:
undelivered_per_month = undelivered.groupby(["months","order_status"],as_index = False).aggregate({"size":"sum"})
undelivered_per_month = undelivered_per_month.groupby("order_status",as_index = False).aggregate({"size":"mean"})

In [80]:
undelivered_per_month["size"][0] = undelivered_per_month["size"][0].round()

/tmp/ipykernel_7453/849065463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  undelivered_per_month["size"][0] = undelivered_per_month["size"][0].round()


In [81]:
undelivered_per_month

,order_status,size
0,canceled,23.0
1,unavailable,29.0


Итого: первая причина недоставки заключается в недоступности товара, а вторая в его отмене. Из-за первой причины в среднем в месяц не доставляется 29 товаров, а из-за второй - 23.

### Ответ на третий вопрос :
____

3. По каждому товару определить, в какой день недели товар чаще всего покупается.

In [82]:
df23 = olist_order_items_dataset.merge(olist_orders_dataset,on = "order_id")
df23.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [83]:
df23['days'] = df23.order_approved_at.dt.day_name()

In [84]:
df23.product_id.unique()

array(['4244733e06e7ecb4970a6e2683c13e61',
       'e5f2d52b802189ee658865ca93d83a8f',
       'c777355d18b72b67abbeef9df44fd0fd', ...,
       'dd469c03ad67e201bc2179ef077dcd48',
       'bbe7651fef80287a816ead73f065fc4b',
       '350688d9dc1e75ff97be326363655e01'], dtype=object)

In [85]:
per_week = df23.query('order_status == "delivered"')\
    .groupby(['product_id','days'], as_index=False)\
    .agg({'price':'count'})

In [86]:
per_week.loc[per_week.groupby('product_id')['price']
    .idxmax()]\
    .sort_values(by='price',ascending=False).rename(columns={"price":"quantity"}).head()

,product_id,days,quantity
40190,aca2eb7d00ea1a7b8ebd4e68314663af,Tuesday,119
19709,53b36df67ebb7c41585e8d54d6772e08,Tuesday,104
15609,422879e10f46682990de24d770e7f83d,Tuesday,89
12793,368c6c730842d78016ad823897a372db,Thursday,80
35778,99a4788cb24856965c36a24e339b6058,Tuesday,78


### Ответ на четвёртый вопрос:
____

Для начала нам стоит снова посмотреть на наш объединённый DataFrame(1 и 2):
Исходя из запроса нам понадобяться всего лишь 2 колонки:
1. customer_unique_id - уникальный id пользователя;
2. order_approved_at - время подтверждения оплаты заказа.

Для начала как всегда проверим и очистим DataFrame, но походу буду придерживаться такого плана:
1. Посчитать сколько наши пользователи покупали каждый месяц каждого года;
2. Посчитать как раз неровное кол-во недель(то есть просто простые недели с оставшимеся днями);
3. Поделить первое на второе.

In [87]:
answer4 = df12[["customer_unique_id","order_approved_at"]].sort_values("order_approved_at")
answer4.head()

,customer_unique_id,order_approved_at
21125,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38
17531,14359ea0c7a105749c0a56478825b015,2016-10-04 09:43:32
59705,b8b8726af116a5cfb35b0315ecef9172,2016-10-04 10:18:57
19876,87776adb449c551e74c13fc34f036105,2016-10-04 10:19:23
21796,88bc832f104b6dd684a17dbb1b76e778,2016-10-04 10:25:46


In [88]:
answer4.dropna(inplace=True)

Сначала найдем месяца каждого года и для этого я просто взял lambda из предыдущего упражнения:

In [89]:
answer4["month_year"] = answer4["order_approved_at"].apply(lambda x:x.strftime("%m-%Y"))
answer4.head()

,customer_unique_id,order_approved_at,month_year
21125,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38,09-2016
17531,14359ea0c7a105749c0a56478825b015,2016-10-04 09:43:32,10-2016
59705,b8b8726af116a5cfb35b0315ecef9172,2016-10-04 10:18:57,10-2016
19876,87776adb449c551e74c13fc34f036105,2016-10-04 10:19:23,10-2016
21796,88bc832f104b6dd684a17dbb1b76e778,2016-10-04 10:25:46,10-2016


Далее создадим небольшой датафрейм, один из столбцов которого как раз и будет отвечать за кол-во покупок каждый месяц каждого года:

In [90]:
quantity_per_month = answer4.groupby(["customer_unique_id","month_year"],as_index=False).aggregate({"order_approved_at":"count"}).\
    rename(columns={"order_approved_at":"quantity_per_month"})

In [91]:
answer4 = answer4.merge(quantity_per_month,how = "inner",on=['customer_unique_id'])#если делать по месяцу у меня умирает консоль, так что делаю по юзерам, а там просто фильрую немного датасет
answer4.drop("month_year_y",inplace=True,axis=1)
answer4 = answer4.rename(columns={"month_year_x":"month_year"})
answer4.head()

,customer_unique_id,order_approved_at,month_year,quantity_per_month
0,830d5b7aaa3b6f1e9ad63703bec97d23,2016-09-15 12:16:38,09-2016,1
1,14359ea0c7a105749c0a56478825b015,2016-10-04 09:43:32,10-2016,1
2,b8b8726af116a5cfb35b0315ecef9172,2016-10-04 10:18:57,10-2016,1
3,87776adb449c551e74c13fc34f036105,2016-10-04 10:19:23,10-2016,1
4,88bc832f104b6dd684a17dbb1b76e778,2016-10-04 10:25:46,10-2016,1


Далее производим подсчёт сколько недель с остатками в каждом месяце:

In [92]:
answer4['days_in_month'] = answer4['order_approved_at'].dt.days_in_month / 7

In [93]:
answer4["mean_orders_per_week"] = answer4["quantity_per_month"] / answer4["days_in_month"]
answer4 = answer4[["customer_unique_id","month_year","mean_orders_per_week"]]
answer4.head()

,customer_unique_id,month_year,mean_orders_per_week
0,830d5b7aaa3b6f1e9ad63703bec97d23,09-2016,0.233333
1,14359ea0c7a105749c0a56478825b015,10-2016,0.225806
2,b8b8726af116a5cfb35b0315ecef9172,10-2016,0.225806
3,87776adb449c551e74c13fc34f036105,10-2016,0.225806
4,88bc832f104b6dd684a17dbb1b76e778,10-2016,0.225806


### Ответ на пятый вопрос:
____

![](cohort.jpg)

Также о том, что такое когорный анализ можно почитать вот [тут](https://vc.ru/s/productstar/134090-chto-takoe-kogortnyy-analiz).

Для начала выберем нужную нам таблицу и приведём все типы данных в порядок:

In [94]:
df12["order_purchase_timestamp"] = df12["order_purchase_timestamp"].astype("str")
df12["order_purchase_timestamp"] = df12["order_purchase_timestamp"].apply(lambda x:x[0:7])
df12["order_purchase_timestamp"] = pd.to_datetime(df12["order_purchase_timestamp"])

Далее создадим колонку с первым месяцем для каждого пользователя:

In [95]:
df12["first_month"] = df12.groupby('customer_unique_id')['order_purchase_timestamp'].transform('min').dt.to_period('M')
df12.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,first_month
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-01,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,2017-05
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-01,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,2018-01
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-01,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,2018-05
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-01,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,2018-03
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-01,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,2018-07


Отберём таблицу, которая нам после понадобится:
1. Дата должна быть с января по декабрь;
2. Переименуем колонки для удобного визуала.

In [96]:
answer5 = df12.query('first_month >= "2017-01" and first_month <= "2017-12"').\
    groupby(['first_month', 'order_purchase_timestamp'], as_index=False).\
    agg({'customer_unique_id':'nunique'}).\
    rename(columns=({'customer_unique_id':'customers',"order_purchase_timestamp":"months_of_purchases"}))
answer5.head()

,first_month,months_of_purchases,customers
0,2017-01,2017-01-01,764
1,2017-01,2017-02-01,3
2,2017-01,2017-03-01,2
3,2017-01,2017-04-01,1
4,2017-01,2017-05-01,3


In [97]:
from operator import attrgetter#берём данный модуль для расчёта разницы периодов

In [98]:
answer5.dtypes

first_month                 period[M]
months_of_purchases    datetime64[ns]
customers                       int64
dtype: object

Так, а теперь приведём колонку с месяцем покупок в порядок, так как attrgetter принимает формат периода:

In [99]:
answer5["months_of_purchases"] = pd.to_datetime(answer5["months_of_purchases"]).dt.to_period("m")

In [100]:
answer5.dtypes

first_month            period[M]
months_of_purchases    period[M]
customers                  int64
dtype: object

Теперь все данные в порядке и давайте посчитаем разницу между датой каждой покупки и первым месяцем:

In [101]:
answer5['months_difference'] = (answer5.months_of_purchases - answer5.first_month).apply(attrgetter('n'))

In [102]:
answer5

,first_month,months_of_purchases,customers,months_difference
0,2017-01,2017-01,764,0
1,2017-01,2017-02,3,1
2,2017-01,2017-03,2,2
3,2017-01,2017-04,1,3
4,2017-01,2017-05,3,4
...,...,...,...,...
166,2017-12,2018-04,15,4
167,2017-12,2018-05,11,5
168,2017-12,2018-06,9,6
169,2017-12,2018-07,3,7


In [103]:
cohort_pivot = answer5.pivot(index='first_month', columns = 'months_difference', values = 'customers')
cohort_pivot.head()

months_difference,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
first_month,,,,,,,,,,,,,,,,,,,
2017-01,764.0,3.0,2.0,1.0,3.0,1.0,4.0,1.0,1.0,NaN,3.0,1.0,6.0,3.0,1.0,1.0,2.0,3.0,1.0
2017-02,1752.0,4.0,5.0,2.0,7.0,2.0,4.0,3.0,3.0,4.0,2.0,5.0,3.0,3.0,2.0,1.0,1.0,4.0,NaN
2017-03,2636.0,13.0,10.0,10.0,9.0,4.0,4.0,8.0,9.0,2.0,10.0,4.0,6.0,3.0,4.0,6.0,2.0,4.0,NaN
2017-04,2352.0,14.0,5.0,4.0,8.0,6.0,8.0,7.0,7.0,4.0,6.0,2.0,2.0,1.0,2.0,2.0,5.0,NaN,NaN
2017-05,3596.0,18.0,18.0,14.0,11.0,12.0,15.0,6.0,9.0,11.0,9.0,12.0,9.0,1.0,7.0,9.0,NaN,NaN,NaN


In [104]:
cohort_size = cohort_pivot.iloc[:,0]
retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)

In [105]:
cohort_final = (retention_matrix
                .style
                .background_gradient(cmap='viridis')  # раскрашиваем ячейки по столбцам
                .highlight_null('white')  # делаем белый фон для значений NaN
                .set_caption('Когортный анализ')
                .format("{:.2%}", na_rep=""))
cohort_final

months_difference,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19
first_month,,,,,,,,,,,,,,,,,,,
2017-01,100.00%,0.39%,0.26%,0.13%,0.39%,0.13%,0.52%,0.13%,0.13%,,0.39%,0.13%,0.79%,0.39%,0.13%,0.13%,0.26%,0.39%,0.13%
2017-02,100.00%,0.23%,0.29%,0.11%,0.40%,0.11%,0.23%,0.17%,0.17%,0.23%,0.11%,0.29%,0.17%,0.17%,0.11%,0.06%,0.06%,0.23%,
2017-03,100.00%,0.49%,0.38%,0.38%,0.34%,0.15%,0.15%,0.30%,0.34%,0.08%,0.38%,0.15%,0.23%,0.11%,0.15%,0.23%,0.08%,0.15%,
2017-04,100.00%,0.60%,0.21%,0.17%,0.34%,0.26%,0.34%,0.30%,0.30%,0.17%,0.26%,0.09%,0.09%,0.04%,0.09%,0.09%,0.21%,,
2017-05,100.00%,0.50%,0.50%,0.39%,0.31%,0.33%,0.42%,0.17%,0.25%,0.31%,0.25%,0.33%,0.25%,0.03%,0.19%,0.25%,,,
2017-06,100.00%,0.48%,0.35%,0.41%,0.25%,0.38%,0.38%,0.22%,0.13%,0.22%,0.32%,0.35%,0.16%,0.13%,0.19%,,,,
2017-07,100.00%,0.51%,0.36%,0.26%,0.28%,0.21%,0.31%,0.10%,0.18%,0.26%,0.23%,0.31%,0.13%,0.26%,,,,,
2017-08,100.00%,0.69%,0.33%,0.26%,0.36%,0.53%,0.29%,0.26%,0.14%,0.14%,0.24%,0.19%,0.10%,,,,,,
2017-09,100.00%,0.68%,0.53%,0.29%,0.46%,0.22%,0.22%,0.24%,0.29%,0.17%,0.27%,0.07%,,,,,,,


Cамый высокий retention на 3 месяц был у когорты 2017-06

### Ответ на шестой вопрос:
____

- Recency (R) as days since last purchase: How many days ago was their last purchase? Deduct most recent purchase date from today to calculate the recency value. 1 day ago? 14 days ago? 500 days ago?
- Frequency (F) as the total number of transactions: How many times has the customer purchased from our store? For example, if someone placed 10 orders over a period of time, their frequency is 10.
- Monetary (M) as total money spent: How many $$ (or whatever is your currency of calculation) has this customer spent? Simply total up the money from all transactions to get the M value.

Честно очень помогло вот это, видимо на англ и вправду лучше все воспринимаю, а также вот хорошая статья:
[Подробная информация](https://medium.com/analytics-vidhya/customer-segmentation-with-rfm-analysis-kmeans-clustering-32c387d04dfe).

In [106]:
from datetime import timedelta #понадобиться потом для симуляции даты наперёд

У нас исторический dataset, поэтому 'сегодняшнею' дату мы будем симулировать, а что на счёт таблиц,  нам понадобяться все:

In [107]:
answer6 = df12.merge(olist_order_items_dataset,on = "order_id")
answer6.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,first_month,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-01,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,2017-05,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-01,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,2018-01,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-01-18 20:58:32,289.00,46.48
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-01,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,2018-05,1,bd07b66896d6f1494f5b86251848ced7,7c67e1448b00f6e969d365cea6b010ab,2018-06-05 16:19:10,139.94,17.79
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-01,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,2018-03,1,a5647c44af977b148e0a3a4751a09e2e,7c67e1448b00f6e969d365cea6b010ab,2018-03-27 16:31:16,149.94,23.36
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-01,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,2018-07,1,9391a573abe00141c56e38d84d7d5b3b,4a3ca9315b744ce9f8e9374361493884,2018-07-31 10:10:09,230.00,22.25


Прежде всего избавить наш dataset от NaN значений, если таковые есть, а также смоделируем наш сегодняшний день(под данным предполагаю плюс 1 день к максимальному дню доставки):

In [108]:
answer6.dropna(inplace=True)

In [109]:
answer6["order_delivered_carrier_date"] = pd.to_datetime(answer6["order_delivered_carrier_date"])

In [110]:
last_date = answer6['order_delivered_carrier_date'].max() + timedelta(days=1)
last_date

Timestamp('2018-09-12 19:48:28')

Далее группируем RFM таблицу, на основе которой мы и бдуем собирать RFM показатель и также я снова же изменил названия столбцов для более удобной наглядности:

In [111]:
rfmTable = answer6.reset_index().groupby('customer_unique_id').agg({'order_delivered_carrier_date': lambda x: (last_date - x.max()).days,
                                                                   'order_id': lambda x: len(x),
                                                                   'price': lambda x: x.sum()})

In [112]:
rfmTable.rename(columns={'order_delivered_carrier_date': 'recency',
                         'order_id': 'frequency',
                         'price': 'monetary_value'}, inplace=True)

In [113]:
rfmTable.head()

,recency,frequency,monetary_value
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,18.90
0000f46a3911fa3c0805444483337064,548,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,333,1,25.99
0004aac84e0df4da2b147fca70cf8255,299,1,180.00


In [114]:
rfmTable.frequency.value_counts()

1     81733
2      8796
3      1646
4       621
5       241
6       190
7        45
8        16
10       11
11       11
9        10
12        9
15        3
14        3
20        2
13        2
18        1
21        1
24        1
Name: frequency, dtype: int64

Дальше мы создаём функции, которые и будут делить наших пользователей по разным показателям RFM:

In [115]:
quantiles = rfmTable.quantile(q=[0.20, 0.40, 0.60, 0.80])
quantiles = quantiles.to_dict()
segmented_rfm = rfmTable

In [116]:
def RScore(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]:
        return 3
    elif x<=d[p][0.80]:
        return 2
    else:
        return 1

In [117]:
def FMScore(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]:
        return 3
    elif x<=d[p][0.80]:
        return 4
    else:
        return 5

In [118]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=("frequency",quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
customer_unique_id,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,129.90,4,1,4,414
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,18.90,4,1,1,411
0000f46a3911fa3c0805444483337064,548,1,69.00,1,1,2,112
0000f6ccb0745a6a4b88665a16c9f078,333,1,25.99,2,1,1,211
0004aac84e0df4da2b147fca70cf8255,299,1,180.00,2,1,5,215


### Выводы :
____


Я ответил на 4 вопроса от бизнеса, а также научился применять когорный анализ и RFM сегментацию.

### Ссылки :
____

1. [Что такое когорный анлиз](https://vc.ru/s/productstar/134090-chto-takoe-kogortnyy-analiz);
2. [Пример когорного анализа на Python](https://towardsdatascience.com/a-step-by-step-introduction-to-cohort-analysis-in-python-a2cbbd8460ea);
3. [RFM](https://aainabajaj39.medium.com/rfm-analysis-for-successful-customer-segmentation-using-python-6291decceb4b).